In [ ]:
from funlib.learn.torch.models import UNet, ConvPass
import gunpowder as gp
import logging
import numpy as np
import torch
import zarr

logging.basicConfig(level=logging.INFO)

checkpoint = 100000
predict_roi = gp.Roi((250, 2000, 500, 500), (1, 160, 1000, 1000))

raw = gp.ArrayKey("RAW")
prediction = gp.ArrayKey("PREDICTION")

input_shape = (7, 64, 124, 124)
output_shape = (1, 32, 32, 32)

unet = UNet(
    in_channels=7,
    num_fmaps=12,
    fmap_inc_factor=5,
    downsample_factors=[
        (1, 2, 2),
        (1, 2, 2),
        (2, 2, 2)],
    constant_upsample=True,
    padding='valid')
model = torch.nn.Sequential(
    unet,
    ConvPass(12, 1, [(1, 1, 1)], activation=None),
    torch.nn.Sigmoid()
)
model.eval()

raw_source = gp.ZarrSource(
    '../data/mouse.n5',
    {raw: 'volumes/raw'},
    {raw: gp.ArraySpec(interpolatable=True)}
)

with gp.build(raw_source):
    print(raw_source)
    voxel_size = raw_source.spec[raw].voxel_size

input_size = voxel_size*input_shape
output_size = voxel_size*output_shape

# TODO: create a predict pipeline and create a prediction for predict_roi!